<a href="https://colab.research.google.com/github/mijin0721/Sentiment_analysis_AI/blob/main/Model_KoBERT_No2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 기본세팅

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 6.5 MB/s 
     |████████████████████████████████| 182 kB 62.3 MB/s 
     |████████████████████████████████| 7.6 MB 58.3 MB/s 


In [ ]:
!pip install keras

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import torch
 
from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras_preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
 
import pandas as pd
import numpy as np
import random
import time
import datetime
 
## GPU 확인
import os
 
n_devices = torch.cuda.device_count()
print('device count:',n_devices)
 
for i in range(n_devices):
    print(i,':',torch.cuda.get_device_name(i))
 
## main 및 test 로드
data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/train_data.csv")

from sklearn.model_selection import train_test_split
train, test = train_test_split(data, test_size=0.2, shuffle=True, random_state=34)

 
print('train shape:',train.shape)
print('test shape:',test.shape)

device count: 1
0 : Tesla T4
train shape: (160000, 3)
test shape: (40000, 3)


In [ ]:
print('sentence processing..')
document_bert = ["[CLS] " + str(s) + " [SEP]" for s in train.review]
document_bert[:5]
print(document_bert[:5])

sentence processing..
['[CLS] 베개유목민인데 복구력이 넘좋다해서 삿지만 너무좋아서 높게느껴질 정도에요 평소 낮은베개를 선호해서 자고일어나면 어깨나 목이아프고 푹 잠을 못잔느낌,,ㅠ [SEP]', '[CLS] 역시 저렴한 이유가 있네요~ 가격에 비해 너무 많은걸 바랬나봐요 딱 가격값하는 엄청 얇은 재질의 상품이에요 [SEP]', '[CLS] 제품은 만족하는데 배송이 한진택배라 엄청 오래걸렸어요 [SEP]', '[CLS] 이건 뭐 쓰던거 파는건가요 뚜껑이 새것처럼 잠금장치가 없네요 물건은 다 샛는지 터졋는지 포장지안은 린스가 차있고 박스도 다 젖어가지고... 포장에돔 신경쓰셔야할듯하네요 [SEP]', '[CLS] 신은지 3주만에 밑창 떨어져서 , 구두방에 만원주고 수선했는데 어차피 금방 또 떨어진다고하시더니 진짜 금방 떨어지네요 [SEP]']


In [ ]:
print('tokenizeing..')
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased',do_lower_case=False)
tokenized_texts = [ tokenizer.tokenize(s) for s in document_bert]
print('tokenized_texts[0]:',tokenized_texts[0])

tokenizeing..


Downloading:   0%|          | 0.00/996k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

tokenized_texts[0]: ['[CLS]', '베', '##개', '##유', '##목', '##민', '##인', '##데', '복', '##구', '##력이', '넘', '##좋', '##다', '##해서', '삿', '##지만', '너', '##무', '##좋', '##아', '##서', '높', '##게', '##느', '##껴', '##질', '정도', '##에', '##요', '평', '##소', '낮', '##은', '##베', '##개', '##를', '선', '##호', '##해서', '자', '##고', '##일', '##어', '##나', '##면', '어', '##깨', '##나', '목', '##이', '##아', '##프', '##고', '[UNK]', '잠', '##을', '못', '##잔', '##느', '##낌', ',', ',', '[UNK]', '[SEP]']


In [ ]:
print('padding')
MAX_LEN = 128
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
input_ids = pad_sequences(input_ids, maxlen = MAX_LEN, dtype='long', truncating='post', padding='post')
input_ids[0]
print('input_ids[0]:',input_ids[0])

padding
input_ids[0]: [   101   9344  21789  42815  68055  36553  12030  28911   9357  17196
  61964   9008 119214  11903  70146   9412  28578   9004  32537 119214
  16985  12424   9028  14153 118760 118684  48599 107657  10530  48549
   9926  22333   8992  10892  92688  21789  11513   9428  20309  70146
   9651  11664  18392  12965  16439  14867   9546 118678  16439   9284
  10739  16985  28396  11664    100   9655  10622   9290 119196 118760
 118713    117    117    100    102      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0]


In [ ]:
attention_masks = []
 
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)
 
print('attention_masks[0]:',attention_masks[0])

attention_masks[0]: [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [ ]:
print('split train - val')
train_inputs, validation_inputs, train_labels, validation_labels = \
train_test_split(input_ids, train['label'].values, random_state=42, test_size=0.1)
 
train_masks, validation_masks, _, _ = train_test_split(attention_masks, input_ids, random_state=42,test_size=0.1)


split train - val


In [ ]:
print('convert data to tenser..')
train_inputs = torch.tensor(train_inputs)
train_labels = torch.tensor(train_labels)
train_masks = torch.tensor(train_masks)
validation_inputs = torch.tensor(validation_inputs)
validation_labels = torch.tensor(validation_labels)
validation_masks = torch.tensor(validation_masks)

convert data to tenser..


In [ ]:
print('set batch and data loader')
BATCH_SIZE = 32
 
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler = train_sampler, batch_size = BATCH_SIZE)
 
validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=BATCH_SIZE)


set batch and data loader


In [ ]:
test

,score,review,label
7388,2,일단 설 끼고도 배송이 엄청느렸어요 그런데 조립 힘들게 다 완성하고 보니 헤드라고 ...,0
10179,1,그냥 허접합니다 색종이로 만든것 같은...,0
27424,2,실망입니다.,0
67371,1,"책상에 고정하니 역방향으로 기울어지고, 팔도 어색합니다. 제 경우에만 그런 것이고 ...",0
137074,4,네이비랑다크그린시켯는데 겉에는네이비라고써져잇는데 제눈엔아무리봐두 둘다다크그린이에여....,1
...,...,...,...
191311,4,촉촉하고 보습력 좋아요,1
148997,5,소재가ㅡ너무좋아요 추가구입의사있어요,1
155359,2,너무 적어요,0
183686,5,하나 더 배송 해주신거 연락은 못받았는데 편의점에 있더라구요 친절히 응해주셔서 감사...,1


In [ ]:
print('data split')
sentences = test['review']
sentences = ["[CLS]"+str(sentence)+"[SEP]" for sentence in sentences]
labels = test['label'].values
 
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
 
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
 
attention_masks = []
for seq in input_ids:
        seq_mask = [float(i>0) for i in seq]
        attention_masks.append(seq_mask)
 
test_inputs = torch.tensor(input_ids)
test_labels = torch.tensor(labels)
test_masks = torch.tensor(attention_masks)
 
test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = RandomSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=BATCH_SIZE)

data split


In [ ]:
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


## 모델링

In [ ]:
model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=2)
model.cuda()

In [ ]:
# 옵티마이저
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # 학습률(learning rate)
                  eps = 1e-8 
                )

# 에폭수
epochs = 4

# 총 훈련 스텝 : 배치반복 횟수 * 에폭
total_steps = len(train_dataloader) * epochs

# 스케줄러 생성
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

In [ ]:
# 정확도 계산 함수
def flat_accuracy(preds, labels):
    
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    return np.sum(pred_flat == labels_flat) / len(labels_flat)
    
    
# 시간 표시 함수
def format_time(elapsed):

    # 반올림
    elapsed_rounded = int(round((elapsed)))
    
    # hh:mm:ss으로 형태 변경
    return str(datetime.timedelta(seconds=elapsed_rounded))


In [ ]:
#랜덤시드 고정
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

#그래디언트 초기화
model.zero_grad()

# 학습
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # 시작 시간 설정
    t0 = time.time()

    # 로스 초기화
    total_loss = 0

    # 훈련모드로 변경
    model.train()
        
    # 데이터로더에서 배치만큼 반복하여 가져옴
    for step, batch in enumerate(train_dataloader):
        # 경과 정보 표시
        if step % 500 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch

        # Forward 수행                
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask, 
                        labels=b_labels)
        
        # 로스 구함
        loss = outputs[0]

        # 총 로스 계산
        total_loss += loss.item()

        # Backward 수행으로 그래디언트 계산
        loss.backward()

        # 그래디언트 클리핑
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # 그래디언트를 통해 가중치 파라미터 업데이트
        optimizer.step()

        # 스케줄러로 학습률 감소
        scheduler.step()

        # 그래디언트 초기화
        model.zero_grad()

    # 평균 로스 계산
    avg_train_loss = total_loss / len(train_dataloader)            

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
        
    # ========================================
    #               Validation
    # ========================================

    print("")
    print("Running Validation...")

    #시작 시간 설정
    t0 = time.time()

    # 평가모드로 변경
    model.eval()

    # 변수 초기화
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # 데이터로더에서 배치만큼 반복하여 가져옴
    for batch in validation_dataloader:
        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch
        
        # 그래디언트 계산 안함
        with torch.no_grad():     
            # Forward 수행
            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        
        # 로스 구함
        logits = outputs[0]

        # CPU로 데이터 이동
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # 출력 로짓과 라벨을 비교하여 정확도 계산
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1

    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))

print("")
print("Training complete!")


======== Epoch 1 / 4 ========
Training...
  Batch   500  of  4,500.    Elapsed: 0:05:42.
  Batch 1,000  of  4,500.    Elapsed: 0:11:23.
  Batch 1,500  of  4,500.    Elapsed: 0:17:06.
  Batch 2,000  of  4,500.    Elapsed: 0:22:48.
  Batch 2,500  of  4,500.    Elapsed: 0:28:30.
  Batch 3,000  of  4,500.    Elapsed: 0:34:12.
  Batch 3,500  of  4,500.    Elapsed: 0:39:54.
  Batch 4,000  of  4,500.    Elapsed: 0:45:36.

  Average training loss: 0.26
  Training epcoh took: 0:51:18

Running Validation...
  Accuracy: 0.91
  Validation took: 0:02:01

======== Epoch 2 / 4 ========
Training...
  Batch   500  of  4,500.    Elapsed: 0:05:42.
  Batch 1,000  of  4,500.    Elapsed: 0:11:24.
  Batch 1,500  of  4,500.    Elapsed: 0:17:06.
  Batch 2,000  of  4,500.    Elapsed: 0:22:48.
  Batch 2,500  of  4,500.    Elapsed: 0:28:30.
  Batch 3,000  of  4,500.    Elapsed: 0:34:12.
  Batch 3,500  of  4,500.    Elapsed: 0:39:54.
  Batch 4,000  of  4,500.    Elapsed: 0:45:36.

  Average training loss: 0.20
  

In [ ]:
#시작 시간 설정
t0 = time.time()

# 평가모드로 변경
model.eval()

# 변수 초기화
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0

# 데이터로더에서 배치만큼 반복하여 가져옴
for step, batch in enumerate(test_dataloader):
    # 경과 정보 표시
    if step % 100 == 0 and not step == 0:
        elapsed = format_time(time.time() - t0)
        print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(test_dataloader), elapsed))

    # 배치를 GPU에 넣음
    batch = tuple(t.to(device) for t in batch)
    
    # 배치에서 데이터 추출
    b_input_ids, b_input_mask, b_labels = batch
    
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)
    
    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    
    # 출력 로짓과 라벨을 비교하여 정확도 계산
    tmp_eval_accuracy = flat_accuracy(logits, label_ids)
    eval_accuracy += tmp_eval_accuracy
    nb_eval_steps += 1

print("")
print("Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
print("Test took: {:}".format(format_time(time.time() - t0)))

  Batch   100  of  1,250.    Elapsed: 0:00:24.
  Batch   200  of  1,250.    Elapsed: 0:00:48.
  Batch   300  of  1,250.    Elapsed: 0:01:12.
  Batch   400  of  1,250.    Elapsed: 0:01:35.
  Batch   500  of  1,250.    Elapsed: 0:02:00.
  Batch   600  of  1,250.    Elapsed: 0:02:24.
  Batch   700  of  1,250.    Elapsed: 0:02:48.
  Batch   800  of  1,250.    Elapsed: 0:03:12.
  Batch   900  of  1,250.    Elapsed: 0:03:36.
  Batch 1,000  of  1,250.    Elapsed: 0:04:00.
  Batch 1,100  of  1,250.    Elapsed: 0:04:25.
  Batch 1,200  of  1,250.    Elapsed: 0:04:49.

Accuracy: 0.92
Test took: 0:05:01


## 테스트

In [ ]:
# 입력 데이터 변환
def convert_input_data(sentences):

    # BERT의 토크나이저로 문장을 토큰으로 분리
    sentences = [sentences]
    tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

    # 입력 토큰의 최대 시퀀스 길이
    MAX_LEN = 128

    # 토큰을 숫자 인덱스로 변환
    input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
    
    # 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
    input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

    # 어텐션 마스크 초기화
    attention_masks = []

    # 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
    # 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
    for seq in input_ids:
        seq_mask = [float(i>0) for i in seq]
        attention_masks.append(seq_mask)

    # 데이터를 파이토치의 텐서로 변환
    inputs = torch.tensor(input_ids)
    masks = torch.tensor(attention_masks)

    return inputs, masks

In [ ]:
# 문장 테스트
def test_sentences(sentences):

    # 평가모드로 변경
    model.eval()

    # 문장을 입력 데이터로 변환
    inputs, masks = convert_input_data(sentences)

    # 데이터를 GPU에 넣음
    b_input_ids = inputs.to(device)
    b_input_mask = masks.to(device)
            
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)

    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()

    return logits

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/test_data.csv', encoding='utf-8')
data

,id,review,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0
...,...,...,...
94,4663025,뭘 만든 건가?,0
95,6753658,전기톱은못들고다니는데 엔진톱이겠죠,0
96,9665771,완전 재밌엇는데 왜 평점이??,1
97,8757576,제임스 완이 내 목표임 ㄷ,1


In [ ]:
v1 = []
for i in range(0, data.shape[0]):
    row = data.iloc[i]
    idx = data.index[i]
    a = row['review']
    v1.append(a)

In [ ]:
v1

['굳 ㅋ',
 'GDNTOPCLASSINTHECLUB',
 '뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아',
 '지루하지는 않은데 완전 막장임... 돈주고 보기에는....',
 '3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??',
 '음악이 주가 된, 최고의 음악영화',
 '진정한 쓰레기',
 '마치 미국애니에서 튀어나온듯한 창의력없는 로봇디자인부터가,고개를 젖게한다',
 '갈수록 개판되가는 중국영화 유치하고 내용없음 폼잡다 끝남 말도안되는 무기에 유치한cg남무 아 그립다 동사서독같은 영화가 이건 3류아류작이다',
 '이별의 아픔뒤에 찾아오는 새로운 인연의 기쁨 But, 모든 사람이 그렇지는 않네..',
 '괜찮네요오랜만포켓몬스터잼밌어요',
 '한국독립영화의 한계 그렇게 아버지가 된다와 비교됨',
 '청춘은 아름답다 그 아름다움은 이성을 흔들어 놓는다. 찰나의 아름다움을 잘 포착한 섬세하고 아름다운 수채화같은 퀴어영화이다.',
 '눈에 보이는 반전이었지만 영화의 흡인력은 사라지지 않았다.',
 '"스토리, 연출, 연기, 비주얼 등 영화의 기본 조차 안된 영화에 무슨 평을 해. 이런 영화 찍고도 김문옥 감독은 ""내가 영화 경력이 몇OO인데 조무래기들이 내 영화를 평론해?"" 같은 마인드에 빠져있겠지?"',
 '소위 ㅈ문가라는 평점은 뭐냐?',
 '최고!!!!!!!!!!!!!!!!',
 '발연기 도저히 못보겠다 진짜 이렇게 연기를 못할거라곤 상상도 못했네',
 '나이스',
 '별 재미도없는거 우려먹어 .... 챔프에서 방송 몇번했더라 ? ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ',
 "'13일의 금요일', '나이트메어'시리즈와 함께 가장 많은 시리즈를 양산해냈던 헬레이저 시리즈의 첫편. 작가의 상상력이 돋보이는 작품이며, 갈고리로 사지찢는 고어씬은 지금보더라도 상당히 잔인하고 충격적이다.",
 '나름 교훈돋기는 하지만 어쩔수없이 저평점 받을수밖에 없는 저질섹스코미디',
 '꽤 재밌게 본 영화였다!',
 '민주화 시

In [ ]:
df_result = []
for i in range(len(v1)):
    logits = test_sentences(v1[i])

    if np.argmax(logits) == 1 :
        d = 1
    elif np.argmax(logits) == 0 :
        d = 0

    df_result.append(d)

In [ ]:
df_result=pd.DataFrame(df_result,columns=['결과'])
df_result.to_excel('/content/drive/MyDrive/result_1DCNN.xlsx')